<h3>Importing the Libraries</h3>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam

import lightning as L
from torch.utils.data import TensorDataset, DataLoader

/home/sum/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


<h3>Creating the LSTM using the in-built nn.LSTM class</h3>

In [2]:
class LightningLSTM(L.LightningModule):
    def __init__(self):
        super().__init__()

        self.lstm = nn.LSTM(input_size = 1, hidden_size = 1)


    def forward(self, input):
        input_trans = input.view(len(input), 1)
        
        lstm_out, temp = self.lstm(input_trans)

        prediction = lstm_out[-1]

        return prediction


    def configure_optimizers(self):
        return Adam(self.parameters(), lr = 0.1)


    def training_step(self, batch, batch_idx):
        #Calculate loss and log the training process
        
        input_i, label_i = batch
        output_i = self.forward(input_i[0])
        loss = (output_i - label_i) ** 2

        self.log("train_loss", loss)

        if label_i == 0:
            self.log("out_0", output_i)
        else:
            self.log("out_1", output_i)

        return loss

<h3>Making Predictions with the Initial Random Values</h3>

In [4]:
model = LightningLSTM()

print("Now let's compare the observed and predicted values:")
print("\nCompany A: Observed = 0, Predicted =",
      model(torch.tensor([0., 0.5, 0.25, 1.])).detach())
print("Company B: Observed = 1, Predicted =",
      model(torch.tensor([1., 0.5, 0.25, 1.])).detach())

Now let's compare the observed and predicted values:

Company A: Observed = 0, Predicted = tensor([-0.1007])
Company B: Observed = 1, Predicted = tensor([-0.1014])


<h3>Training the Model</h3>

In [5]:
inputs = torch.tensor([[0., 0.5, 0.25, 1.], [1., 0.5, 0.25, 1.]])
labels = torch.tensor([0., 1.])     #Expected (actual) outputs for Companies A & B

dataset = TensorDataset(inputs, labels)
dataloader = DataLoader(dataset)

trainer = L.Trainer(max_epochs = 300, log_every_n_steps = 2)
trainer.fit(model, train_dataloaders = dataloader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
2024-07-03 19:59:09.810118: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-03 19:59:09.826212: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-03 19:59:09.848743: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-03 19:59:09.848807: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuB

Epoch 299: 100%|██████████| 2/2 [00:00<00:00, 125.28it/s, v_num=1]

`Trainer.fit` stopped: `max_epochs=300` reached.


Epoch 299: 100%|██████████| 2/2 [00:00<00:00, 97.26it/s, v_num=1] 


<h3>Making Predictions After Training the Model</h3>

In [6]:
print("Now let's compare the observed and predicted values:")
print("\nCompany A: Observed = 0, Predicted =",
      model(torch.tensor([0., 0.5, 0.25, 1.])).detach())
print("Company B: Observed = 1, Predicted =",
      model(torch.tensor([1., 0.5, 0.25, 1.])).detach())

Now let's compare the observed and predicted values:

Company A: Observed = 0, Predicted = tensor([1.2646e-06])
Company B: Observed = 1, Predicted = tensor([0.9822])
